In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



Let's retrieve all the shortlists for each user and put into a single dataframe


In [ ]:
filelist = []
for i in range(1, 11):
    filelist.append("../input/user%d-venue-shortlist/shortlist_user%d.csv" % (i,i))
filelist

In [ ]:
combinedf = pd.concat([pd.read_csv(f) for f in filelist ])
combinedf.shape

Prepare a dataframe with just user_id, venue_id, and ratings column initialised to 0

In [ ]:
predictiondf = combinedf[['user_id', 'id']]
predictiondf['ratings'] = 0
predictiondf.head()

Now we need to pass this dataframe into our saved SVD model for it to make predictions

In [ ]:
from surprise import Reader, Dataset, SVD
reader = Reader()
data = Dataset.load_from_df(predictiondf, reader)
testset = data.build_full_trainset()



Load saved model


In [ ]:
from surprise import dump
file_name = '../input/recommendation-model-training/movie_svd_model'
_, loaded_algo = dump.load(file_name)

Run predictions

In [ ]:
predictions = loaded_algo.test(testset.build_testset())



Define a helper function to get top n recommendations


In [ ]:
from collections import defaultdict
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])